<a href="https://colab.research.google.com/github/elhamod/BA820/blob/main/Association_Rules/Basic_Association_Rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Course: BA820 - Unsupervised and Unstructured ML**

**Notebook created by: Mohannad Elhamod**

#Analyzing Trends at a Grocery Store.

In this notebook, we will use `mlxtend` to mine association rules from grocery transactions.

In [9]:
# supressing some unimportant warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [10]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

## Load the data

In [11]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "GroceryStoreDataSet.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "simgeerek/grocerystoredataset",
  file_path,
  pandas_kwargs={"header": None},   # This is the additonal line
)

# replace "COCK" with "COKE"
df = df.apply(lambda x: x.str.replace("COCK", "COKE"))
df = df.apply(lambda x: x.str.replace("SUGER", "SUGAR"))

display(df)

Using Colab cache for faster access to the 'grocerystoredataset' dataset.


,0
0,"MILK,BREAD,BISCUIT"
1,"BREAD,MILK,BISCUIT,CORNFLAKES"
2,"BREAD,TEA,BOURNVITA"
3,"JAM,MAGGI,BREAD,MILK"
4,"MAGGI,TEA,BISCUIT"
5,"BREAD,TEA,BOURNVITA"
6,"MAGGI,TEA,CORNFLAKES"
7,"MAGGI,BREAD,TEA,BISCUIT"
8,"JAM,MAGGI,BREAD,TEA"
9,"BREAD,MILK"


## Preprocessing

Convert the text in the table to a list of items

In [12]:
data_column = df.iloc[:, 0]
data = list(data_column.apply(lambda x: x.split(',')))
data

[['MILK', 'BREAD', 'BISCUIT'],
 ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['MAGGI', 'TEA', 'CORNFLAKES'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK'],
 ['COFFEE', 'COKE', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'COKE', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGAR', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'COKE'],
 ['BREAD', 'SUGAR', 'BISCUIT'],
 ['COFFEE', 'SUGAR', 'CORNFLAKES'],
 ['BREAD', 'SUGAR', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'SUGAR'],
 ['BREAD', 'COFFEE', 'SUGAR'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]

We need to convert the data into an encoded format.

In [13]:
# Transform data
te = TransactionEncoder()
transactions = te.fit(data).transform(data) # or fit_transform(data)

# Create a dataframe from the data
df_encoded = pd.DataFrame(transactions, columns=te.columns_)
df_encoded

,BISCUIT,BOURNVITA,BREAD,COFFEE,COKE,CORNFLAKES,JAM,MAGGI,MILK,SUGAR,TEA
0,True,False,True,False,False,False,False,False,True,False,False
1,True,False,True,False,False,True,False,False,True,False,False
2,False,True,True,False,False,False,False,False,False,False,True
3,False,False,True,False,False,False,True,True,True,False,False
4,True,False,False,False,False,False,False,True,False,False,True
5,False,True,True,False,False,False,False,False,False,False,True
6,False,False,False,False,False,True,False,True,False,False,True
7,True,False,True,False,False,False,False,True,False,False,True
8,False,False,True,False,False,False,True,True,False,False,True
9,False,False,True,False,False,False,False,False,True,False,False


## Association Rule Extraction

Let's find some frequent itemsets.

In [14]:
frequent_itemsets = apriori(df_encoded, min_support=0.000001, use_colnames=True)
frequent_itemsets.sort_values(by="support")

,support,itemsets
24,0.05,"(COKE, BREAD)"
25,0.05,"(CORNFLAKES, BREAD)"
20,0.05,"(COFFEE, BOURNVITA)"
17,0.05,"(SUGAR, BISCUIT)"
60,0.05,"(COFFEE, COKE, BREAD)"
...,...,...
5,0.30,(CORNFLAKES)
0,0.35,(BISCUIT)
10,0.35,(TEA)
3,0.40,(COFFEE)


Let's find rules with certain criteria

In [15]:
rules = association_rules(frequent_itemsets,
                          num_itemsets=frequent_itemsets.shape[0],
                          metric="confidence", min_threshold=0.6) #, metric="support", min_threshold=0.05
rules.sort_values(by=["support", "confidence"])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
12,"(CORNFLAKES, BREAD)",(BISCUIT),0.05,0.35,0.05,1.000000,2.857143,1.0,0.0325,inf,0.684211,0.142857,1.000000,0.571429
14,"(SUGAR, BISCUIT)",(BREAD),0.05,0.65,0.05,1.000000,1.538462,1.0,0.0175,inf,0.368421,0.076923,1.000000,0.538462
29,"(COFFEE, BOURNVITA)",(SUGAR),0.05,0.30,0.05,1.000000,3.333333,1.0,0.0350,inf,0.736842,0.166667,1.000000,0.583333
30,"(COKE, BREAD)",(COFFEE),0.05,0.40,0.05,1.000000,2.500000,1.0,0.0300,inf,0.631579,0.125000,1.000000,0.562500
32,"(CORNFLAKES, BREAD)",(MILK),0.05,0.25,0.05,1.000000,4.000000,1.0,0.0375,inf,0.789474,0.200000,1.000000,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,(SUGAR),(BREAD),0.30,0.65,0.20,0.666667,1.025641,1.0,0.0050,1.05,0.035714,0.266667,0.047619,0.487179
7,(CORNFLAKES),(COFFEE),0.30,0.40,0.20,0.666667,1.666667,1.0,0.0800,1.80,0.571429,0.400000,0.444444,0.583333
8,(SUGAR),(COFFEE),0.30,0.40,0.20,0.666667,1.666667,1.0,0.0800,1.80,0.571429,0.400000,0.444444,0.583333
4,(MILK),(BREAD),0.25,0.65,0.20,0.800000,1.230769,1.0,0.0375,1.75,0.250000,0.285714,0.428571,0.553846


Let's filter the rules further.

In [16]:
rules_filtered = rules[(rules['confidence'] > 0.5) & (rules['lift'] >= 3)]
rules_filtered.sort_values(by=["confidence", "lift"])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
43,(COKE),"(CORNFLAKES, COFFEE)",0.15,0.20,0.10,0.666667,3.333333,1.0,0.0700,2.40,0.823529,0.400000,0.583333,0.583333
22,"(CORNFLAKES, BISCUIT)",(COKE),0.15,0.15,0.10,0.666667,4.444444,1.0,0.0775,2.55,0.911765,0.500000,0.607843,0.666667
24,(COKE),"(CORNFLAKES, BISCUIT)",0.15,0.15,0.10,0.666667,4.444444,1.0,0.0775,2.55,0.911765,0.500000,0.607843,0.666667
66,"(CORNFLAKES, BISCUIT)","(COFFEE, COKE)",0.15,0.15,0.10,0.666667,4.444444,1.0,0.0775,2.55,0.911765,0.500000,0.607843,0.666667
67,"(COFFEE, COKE)","(CORNFLAKES, BISCUIT)",0.15,0.15,0.10,0.666667,4.444444,1.0,0.0775,2.55,0.911765,0.500000,0.607843,0.666667
18,(COKE),"(COFFEE, BISCUIT)",0.15,0.10,0.10,0.666667,6.666667,1.0,0.0850,2.70,1.000000,0.666667,0.629630,0.833333
34,"(MAGGI, BREAD)",(JAM),0.15,0.10,0.10,0.666667,6.666667,1.0,0.0850,2.70,1.000000,0.666667,0.629630,0.833333
70,(COKE),"(CORNFLAKES, COFFEE, BISCUIT)",0.15,0.10,0.10,0.666667,6.666667,1.0,0.0850,2.70,1.000000,0.666667,0.629630,0.833333
20,"(COFFEE, BISCUIT)",(CORNFLAKES),0.10,0.30,0.10,1.000000,3.333333,1.0,0.0700,inf,0.777778,0.333333,1.000000,0.666667
23,"(COKE, BISCUIT)",(CORNFLAKES),0.10,0.30,0.10,1.000000,3.333333,1.0,0.0700,inf,0.777778,0.333333,1.000000,0.666667


## Questions:

* Compute basic descriptive statistics for the dataset, such as the average number of items per transaction and the 10 most frequent itemsets.

* The store's online platform aims to recommend a single additional item once a customer has added three items to their cart. Display and interpret the association rules that support such recommendations.

* The store has an excess inventory of tea that is approaching its expiration date and wants to sell it quickly. Based on your analysis, which customer segments should tea be promoted to, and why?

* Empirically demonstrate how the computational complexity (i.e., runtime) of your code changes as a function of:
  1. the number of transactions, and/or
  2. the filtering criteria (e.g., minimum support or confidence thresholds).
You may use tools such as `%%timeit` to support your analysis.

